In [1]:
import os
import re
import pandas as pd
from typing import Optional
from dotenv import load_dotenv
from pydantic import BaseModel, Field

load_dotenv()


True

In [2]:
from langchain_core.tools import tool
from langchain_neo4j import Neo4jGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputParser, PydanticOutputParser
from langchain.output_parsers import OutputFixingParser


In [4]:
graph = Neo4jGraph(refresh_schema=False, url="bolt://localhost:7687", username="neo4j", password="fraud_detection")


In [6]:
llm = ChatGoogleGenerativeAI(
    api_key= os.getenv("GOOGLE_API_KEY"),
    model="gemini-1.5-flash"
)


In [8]:
class Response(BaseModel):
    step: str = Field(description="Current step name")
    content:str = Field(description="A response of the current step.", default="")
    tool_name: Optional[str] = Field(description="Name of the tool that has to call.", default=None)
    tool_args: Optional[str] = Field(description="Arguments or parameters for the tool.", default=None)
    tool_res: Optional[str] = None
    
parser = PydanticOutputParser(pydantic_object=Response)


In [10]:
@tool
def csv_details(csv_file: str) -> str:
    """Returns CSV file starting 5 rows with column names"""
    data = pd.read_csv(f"../datasets/{csv_file}")
    return data.head().to_csv(index=False)



In [11]:
r = await csv_details.ainvoke("test.csv")
print(r)


person,age,company,city
Alice,28,NeoTech,New York
Bob,32,CyberCorp,San Francisco
Charlie,28,NeoTech,New York
Dave,45,HealthPlus,Chicago
Eve,28,CyberCorp,San Francisco



In [15]:
@tool
async def cql_executor(cql: str) -> str:
    """Executes Cypher query on Neo4j and returns results or error message."""
    
    try:
        graph.query(f"""{cql}""")
        return f"✅ Sucessfully Executed";
    except Exception as e:
        return f"❌ FAILED: {str(e)}";



In [19]:
# r = await cql_executor.ainvoke("MERGE (p:Person {name: 'Sourav'}) RETURN p;")


In [20]:
tools_list = [csv_details, cql_executor]


In [21]:
tools = {}
for tl in tools_list:
    tools[tl.name] = {
        "description": tl.description,
        "parameters": tl.args,
        "fn": tl
    }
print(tools)


{'csv_details': {'description': 'Returns CSV file starting 5 rows with column names', 'parameters': {'csv_file': {'title': 'Csv File', 'type': 'string'}}, 'fn': StructuredTool(name='csv_details', description='Returns CSV file starting 5 rows with column names', args_schema=<class 'langchain_core.utils.pydantic.csv_details'>, func=<function csv_details at 0x000001D637B62A20>)}, 'cql_executor': {'description': 'Executes Cypher query on Neo4j and returns results or error message.', 'parameters': {'cql': {'title': 'Cql', 'type': 'string'}}, 'fn': StructuredTool(name='cql_executor', description='Executes Cypher query on Neo4j and returns results or error message.', args_schema=<class 'langchain_core.utils.pydantic.cql_executor'>, coroutine=<function cql_executor at 0x000001D637BBCC20>)}}


In [22]:
def escape_braces(obj):
    string_repr = str(obj)
    escaped = string_repr.replace('{', '{{').replace('}', '}}')
    return escaped


In [23]:
tools_definition = escape_braces(tools)
# tools_definition


In [24]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """
        You are an expert graph database engineer. You are expert in writing Cypher Query Language(CQL).
        
        #Instructions
        You takes user data, understand their data, and then writes cypher query language to insert that data in a graph database.
        You properly categorize data into entities and then start creating relationship between them.
        You works in a flow that, think -> research -> entities -> relationships -> cql -> observe -> output.
        You use your tools according to situations when needed.
        You are allowed to use your tool anytime in-between the flow. 
        
        #Rules
        You perform only one step at a time.
        You execute only one step of a flow at a time.
        You can use your tool anytime in between the flow. 
        
        #Format Instructions
        {format_instructions}
        
        Here is your tools list:
        {tools}
        
        #Example
        - Input - Store the given csv file data in the graph database.
                File = people_company_city.csv
        - Output: {{ 'step': 'think', 'content': 'Okay, user want to store the given file data in a graph database. To store that data I have to collect information about that and understand that. To get the information about that csv file, I should use my tool.' }}
        - Output - {{ 'step': 'tool', 'content': '', 'tool_name': 'csv_details', 'tool_args': 'people_company_city.csv'}}
        - Output - {{ 'step': 'tool_res', 'content': 'person,age,company,city\\r\\nAlice,28,NeoTech,New York\\r\\nBob,32,CyberCorp,San Francisco\\r\\nCharlie,28,NeoTech,New York\\r\\nDave,45,HealthPlus,Chicago\\r\\nEve,28,CyberCorp,San Francisco\\r\\n'}}
        - Output:  {{ 'step': 'research', 'content': 'I can see that the csv is about the people who works in different companies in different cities.' }}
        - Output: {{ 'step': 'entities', 'content': 'Now, I have to find entities. I can see from my research that there are total 4 entities, i.e.,  Person, Age, Company and City.' }}
        - Output: {{ 'step': 'relationships', 'content': 'Now, I have to make relationships between entities. Since, I have already found the entities and I also have the starting data of the CSV from my research, so now I can easily make the relationships between them. 
        Relationships:
         Person - [:is_of] -> Age
         Person - [:works_in] -> Company
         Person - [:lives_in] -> City
         Company - [:located_in] -> City' }} 
        - Output: {{ 'step': 'cql', 'content': 'Now, I have to write cypher query language to store the csv into the graph db. Since, I have already found the entites and the relationships between them, now I can easily write the cql for them.
        CQL:
         LOAD CSV WITH HEADERS
         FROM "file:///people_company_city.csv" AS row
 
         MERGE (p:Person {{name: row.person}})
         SET p.age = toInteger(row.age)
 
         MERGE (co:Company {{name: row.company}})
         MERGE (ci:City {{name: row.city}})
 
         MERGE (p)-[:WORKS_AT]->(co)
         MERGE (p)-[:LIVES_IN]->(ci)
         MERGE (co)-[:LOCATED_IN]->(ci);
        I should use my tool to execute this cql. 
        ' }}
        - Output - {{ 'step': 'tool', 'content': '', 'tool_name': 'cql_executor', 'tool_args': 'LOAD CSV WITH HEADERS
         FROM "file:///people_company_city.csv" AS row
 
         MERGE (p:Person {{name: row.person}})
         SET p.age = toInteger(row.age)
 
         MERGE (co:Company {{name: row.company}})
         MERGE (ci:City {{name: row.city}})
 
         MERGE (p)-[:WORKS_AT]->(co)
         MERGE (p)-[:LIVES_IN]->(ci)
         MERGE (co)-[:LOCATED_IN]->(ci);' }}
        - Output - {{ 'step': 'tool_res', 'content': '✅ Executed successfully.'}} 
        - Output: {{ 'step': 'observe', 'content': 'Now, it is looking that cql has been written and executed successfully according to the tool response. So now I am ready to give the final output to the user.' }}
        - Output: {{ 'step': 'result', 'content': 'Your people_company_city.csv file has successfully stored to the graph database.' }}
        
        **IMPORTANT NOTE** - Do only 1 step at a time of flow as shown in example.
    """),
    MessagesPlaceholder("history"),
    ("human", "{query}"),
])


In [25]:
history = []


In [26]:
new_parser = OutputFixingParser.from_llm(llm=llm, parser=JsonOutputParser())


In [27]:
chain = prompt | llm | new_parser


In [29]:
while True:
    results = chain.invoke(input={
        "history": history,
        "query": "synthetic_bank_transactions.csv",
        "tools": tools_definition,
        "format_instructions": parser.get_format_instructions()
    })

    print(results)
    print("*"*100);
    
    if results == None:
        break;
    
    if results.get("step") == "result":
        break;
    
    history.append(("ai", escape_braces(results)));
    
    if results.get("step") == "tool":
        tool_name = results.get("tool_name")
        tool_args = results.get("tool_args")
        if tool_name in tools:
            tool_res = await tools.get(tool_name).get("fn").ainvoke(tool_args)
            tool_res = {"step": "tool_res", "content": tool_res}
            print(tool_res);
            print("*"*100);
            history.append(("ai", escape_braces(tool_res)))



{'step': 'think', 'content': "The user wants to store data from the synthetic_bank_transactions.csv file into a graph database.  I need to understand the data's structure and relationships to design an appropriate Cypher query.  To do this, I will first use a tool to examine the CSV file's contents."}
****************************************************************************************************
{'step': 'tool', 'content': '', 'tool_name': 'csv_details', 'tool_args': 'synthetic_bank_transactions.csv', 'tool_res': None}
****************************************************************************************************
{'step': 'tool_res', 'content': 'transaction_id,timestamp,amount,currency,sender_account_id,receiver_account_id,sender_customer_id,receiver_customer_id,transaction_type,sender_country,receiver_country\r\n1,2025-06-04T08:32:11,89290.85,INR,ACC1005,ACC1079,CUST024,CUST028,payment,China,India\r\n2,2025-06-05T20:54:25,53829.7,INR,ACC1093,ACC1043,CUST029,CUST007,transfer,

In [ ]:
history.pop()


In [ ]:
history
